---
comments: true
layout: notebook
title: Class 3- JWT Implimentation
description: Teamteach by Lindsay, Anika, Grace, Samhita
type: collab
courses: { csa: {week: 19} }
---

# JWT Implimentation

![JWT](https://files.catbox.moe/t69vm3.png)

## What is a JWT (JSON Web Token)?

- JSON Web Tokens (JWT) serve as a secure means of transmitting information in JSON format, providing a way to establish trust between different parties in a web application. 
- Typically used to transmit information about an authenticated user and additional metadata (usernames, user ID, email, roles, etc.).

### Parts of a JWT:

![Parts](https://files.catbox.moe/19taua.png)

- Each section is divided by a dot
- Header (red): Contains information about how the JWT is encoded, such as the type of token (JWT) and the signing algorithm being used.
- Payload (purple): Contains the claims. Claims are statements about an entity (typically, the user) and additional data. There are three types of claims: registered, public, and private claims.
- Signature (teal): Created by combining the encoded header, encoded payload, and a secret (or key) using the specified algorithm. This signature is used to verify that the sender of the JWT is who it says it is and to ensure that the message wasn't changed along the way.

### HMAC and RSA or ECDSA Keys:

- Used for signing JWT tokens
- HMAC Algorithm (Secret Key): 
  - HMAC (Hash-based Message Authentication Code)
  - A secret key is used to both create and verify the signature.
  - The issuer (who creates the JWT) uses a secret key and a hashing algorithm to create a signature by combining the encoded header and payload.
  - The recipient (who validates the JWT) uses the same secret key to independently calculate the signature and verify that it matches the one provided in the JWT.
  - Symmetric key solution: same key used for signing and verification
- RSA or ECDSA (Public/Private Key Pair):
  - RSA (Rivest–Shamir–Adleman) or ECDSA (Elliptic Curve Digital Signature Algorithm)
  - A pair of keys is used: a private key for signing and a corresponding public key for verification.
  - The issuer signs the JWT using their private key, and the signature is included in the JWT.
  - The recipient, who wants to verify the JWT, uses the public key associated with the issuer to check the signature's validity.
  - Asymmetric key solution: private key is kept secret, and only the public key is shared.

### Well Suited for APIs:

- Compact Format: JWTs are compact in size, as they are encoded in a way that minimizes the amount of data transmitted. This is particularly beneficial in web environments where minimizing data transfer is important for efficiency.
- URL-Safe Encoding: JWTs are encoded using URL-safe Base64, which means that the resulting token can be safely included in URLs without the need for additional encoding. This makes them convenient for passing tokens as part of URLs in web applications.
- Self-Contained: A JWT contains all the necessary information within the token itself. The claims, which include information about the user and additional metadata, are embedded in the token. This eliminates the need for the recipient to make additional database queries or API calls to validate or retrieve information associated with the token.
- Statelessness: JWTs are stateless, meaning that the server does not need to store session information. The necessary information is included in the token itself, allowing the server to validate and process the token without relying on a centralized session store.
- Ease of Integration: JWTs are widely supported and can be easily integrated into various programming languages and frameworks. This interoperability makes them a popular choice for authentication and authorization mechanisms in web APIs.



## Implimenting JWT in Flask

#### 1) Install Flas-JWT-Extended

In [ ]:
pip install Flask-JWT-Extended

#### 2) Include JWT Library in Flask App Setup

In [ ]:
from flask_jwt_extended import JWTManager

# You don't have to add this again if you already have it.
app = Flask(__name__)

# Setup the Flask-JWT-Extended extension
app.config["JWT_SECRET_KEY"] = "secret"  # Remember to change "secret" to a more complex key
jwt = JWTManager(app)

#### 3) Create an Endpoint for Generating Tokens

Using POST because it is creating tokens (POST is for creation)

In [ ]:
POST /token
Content-type: application/json
Body:
{
     "username": "alesanchezr",
     "password": "12341234"
}

In [ ]:
from flask_jwt_extended import create_access_token

# Create a route to authenticate your users and return JWT Token
# The create_access_token() function is used to actually generate the JWT
@app.route("/token", methods=["POST"])
def create_token():
    username = request.json.get("username", None)
    password = request.json.get("password", None)

    # Query your database for username and password
    user = User.query.filter_by(username=username, password=password).first()

    if user is None:
        # The user was not found on the database
        return jsonify({"msg": "Bad username or password"}), 401
    
    # Create a new token with the user id inside
    access_token = create_access_token(identity=user.id)
    return jsonify({ "token": access_token, "user_id": user.id })

#### 4) Use @jwt_required() Decorator on Private Routes

Endpoints that require authorization (private endpoints) should use the @jwt_required() decorator. You can retrieve valid, authenticated user information using the get_jwt_identity function. 